# Reinforcement learning: Policies and value functions valuation
In this first notebook, it will be created the code to interactive policies and value functions evaluate. First, it will be created a Grid object for the game.

## Gridworld¶
The Gridworld game is a board in which an agent can move until reaching one of two final states.

The board and the game can be controlled with the `Grid` object.

In [12]:
import numpy as np


class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]

    # Class properties
    possible_actions = ('U', 'D', 'L', 'R')

    def set(self, rewards, actions):
        # rewards is a dictionary: (i, j): r (row, col): rewards
        # actions is a dictionary: (i, j): A (row, col): list of possible actions

        self.rewards = rewards
        self.actions = actions

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def get_actions(self):
        return self.actions[(self.i, self.j)]

    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions

    def move(self, action):
        # Check if the move is legal valid
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1

        # Returns the reward
        return self.rewards.get((self.i, self.j), 0)

    def undo_move(self, action):
        # Undo the movement
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1

        # Throw an exception in the cell in invalid.
        assert(self.current_state() in self.all_states())

    def game_over(self):
        # Returns true when the game is over and false otherwise.
        return (self.i, self.j) not in self.actions

    def all_states(self):
        # Returns all states
        return set(self.actions.keys()) | set(self.rewards.keys())

To facilitate future work it will create a function that can create a board with the game and the possible cost.

In [3]:
def create_grid(step_cost=0):
    # Create a new board with the reward and possible actions in the cells.
    # Default each movement does not have a cost.
    #
    # The board is as follows
    #   S initial point
    #   X not allowed cell
    #   . allowed cell
    #
    # Number indicate the rewards of states
    #
    #
    # .  .  .  1
    # .  x  . -1
    # s  .  .  .

    grid = Grid(3, 4, (2, 0))

    rewards = {
        (0, 0): step_cost,
        (0, 1): step_cost,
        (0, 2): step_cost,
        (0, 3): 1,
        (1, 0): step_cost,
        (1, 2): step_cost,
        (1, 3): -1,
        (2, 0): step_cost,
        (2, 1): step_cost,
        (2, 2): step_cost,
        (2, 3): step_cost}

    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U')
    }

    grid.set(rewards, actions)

    return grid

Another interesting function are print values, print policy and print value and policy.

In [13]:
def print_values(value, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            v = value.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")


def print_policy(policy, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            action = policy.get((i, j), ' ')

            if action == 'U':
                print("  ↑  |", end="")
            elif action == 'D':
                print("  ↓  |", end="")
            elif action == 'R':
                print("  →  |", end="")
            elif action == 'L':
                print("  ←  |", end="")
            else:
                print("     |", end="")
        print("")


def print_value_policy(V, policy, grid):
    print("Value function")
    print_values(V, grid)
    print()
    print("Policy")
    print_policy(policy, grid)

Finally, a function to initializes the states of the value function randomly or include an input except for the final states. Final states witch are always zero.

In [5]:
def init_states(grid, value=None):
    states = grid.all_states()
    V = {}

    for s in states:
        if s in grid.actions:
            if value is None:
                V[s] = np.random.random()
            else:
                V[s] = value
        else:
            V[s] = 0

    return V, states

## Iterative evaluation of value function and policy 
Now value functions and policies iteratively can be evaluated iteratively. For this, the Bellman equation will be applied until the results converge.

### Value function
The value function of the game can be calculated assuming that the decision to be taken in each of the movements is random. For this it is possible to use an iterative method.

In [14]:
grid = create_grid()
V, states = init_states(grid)

num_iter = 0
gamma = 1
max_iter = 100
threshold = 1e-3

# Iterate until the convergence criterion is verified or maximum iterations is archives.
while num_iter < max_iter:
    num_iter += 1
    biggest_change = 0

    # Iterate over all states
    for s in states:
        old_v = V[s]

        # It is only calculated if there is a policy for the state
        if s in grid.actions:
            new_v = 0
            p_a = 1.0 / len(grid.actions[s])

            for a in grid.actions[s]:
                grid.set_state(s)
                r = grid.move(a)
                new_v += p_a * (r + gamma * V[grid.current_state()])

            V[s] = new_v

            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    if biggest_change < threshold:
        break

if num_iter >= max_iter:
    print("The maximum number of iterations has been reached")

print_values(V, grid)

---------------------------
-0.03| 0.09| 0.22| 0.00|
---------------------------
-0.16| 0.00|-0.44| 0.00|
---------------------------
-0.29|-0.41|-0.54|-0.77|


### Policy
The process can be done analogously with the policies. In this case, it is necessary to define a policy and solve the Bellman equation in an iterative way for the cells where there is a defined policy.

First, the policy to be evaluated must be defined

In [15]:
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'R',
    (2, 1): 'R',
    (2, 2): 'R',
    (2, 3): 'U'}

print_policy(policy, grid)

---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  →  |     |
---------------------------
  ↑  |  →  |  →  |  ↑  |


Now a process similar to the previously one is implemented.

In [16]:
V, states = init_states(grid)

num_iter = 0

# Iterate until the convergence criterion is verified or maximum iterations is archives.
while num_iter < max_iter:
    num_iter += 1
    biggest_change = 0

    # Iterate over all states
    for s in states:
        old_v = V[s]

        # It is only calculated if there is a policy for the state
        if s in policy:
            a = policy[s]
            grid.set_state(s)
            r = grid.move(a)
            V[s] = r + gamma * V[grid.current_state()]

            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    if biggest_change < threshold:
        break

if num_iter >= max_iter:
    print("The maximum number of iterations has been reached")

print_values(V, grid)

---------------------------
 1.00| 1.00| 1.00| 0.00|
---------------------------
 1.00| 0.00|-1.00| 0.00|
---------------------------
 1.00|-1.00|-1.00|-1.00|


It is possible to note that any cell that arrives at the positive final states has a value equal to 1 and -1 where it arrives at the negative final state. This is because without a discount rate it does not matter how long it takes to reach the final state.

### Code factorization
It is a good practice to factorize repeated code. Policy evaluation will be used multiple times, so it can create the following function.

In [17]:
def policy_evaluation(grid, policy, V, states, gamma=1, max_iter=100, threshold=1e-3):
    num_iter = 0

    while num_iter < max_iter:
        num_iter += 1
        biggest_change = 0
        for s in states:
            old_v = V[s]

            if s in policy:
                action = policy[s]
                grid.set_state(s)
                r = grid.move(action)
                V[s] = r + gamma * V[grid.current_state()]
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))

        if biggest_change < threshold:
            break

    if num_iter >= max_iter:
        print("The maximum number of iterations has been reached")

In this function it is not necessary to return the value of `V`, since this is an object and therefore it is passed by reference.

Now the results obtained previously can be checked again.

In [18]:
V, states = init_states(grid)
policy_evaluation(grid, policy, V, states)

print_values(V, grid)

---------------------------
 1.00| 1.00| 1.00| 0.00|
---------------------------
 1.00| 0.00|-1.00| 0.00|
---------------------------
 1.00|-1.00|-1.00|-1.00|


It is also possible check that indicating a discount rate the values in the value function will change.

In [19]:
V, states = init_states(grid)
policy_evaluation(grid, policy, V, states, gamma=0.9)

print_values(V, grid)

---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00|-1.00| 0.00|
---------------------------
 0.66|-0.81|-0.90|-1.00|


Or when the rewards are changed. The rewards that are in use are

In [20]:
print_values(grid.rewards, grid)

---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


But it can be changed to penalize extra movements using a negative reward of 0.1.

In [12]:
grid = create_grid(step_cost=-0.1)
print_values(grid.rewards, grid)

---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In this case the value function of the policy becomes

In [13]:
V, states = init_states(grid)

policy_evaluation(grid, policy, V, states, gamma=0.9)

print_values(V, grid)

---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00|-1.00| 0.00|
---------------------------
 0.31|-1.00|-1.00|-1.00|


## Policy optimization
Based on the function that evaluates the policy it is possible to create a new function to optimize. For that, it is necessary to iterate over the functions to get the best policy.

In [14]:
def optimal_policy(grid, policy, gamma=1, max_iter=100, threshold=1e-3):

    # Initialize the funciton value and states
    V, states = init_states(grid)

    # Iterate over policies until convergence or maximum iterations
    num_iter = 0

    while num_iter < max_iter:
        num_iter += 1

        # Policy evaluation
        policy_evaluation(grid, policy, V, states, gamma, max_iter, threshold)

        # Improvement the policy
        is_policy_converged = True
        for s in states:
            if s in policy:
                old_a = policy[s]
                new_a = None
                best_value = float('-inf')

                # Iterate over actions until the best is obtained
                for a in Grid.possible_actions:
                    grid.set_state(s)
                    r = grid.move(a)
                    v = r + gamma * V[grid.current_state()]
                    if v > best_value:
                        best_value = v
                        new_a = a
                policy[s] = new_a

                if new_a != old_a:
                    is_policy_converged = False

        if is_policy_converged:
            break

    if num_iter >= max_iter:
        print("The maximum number of iterations has been reached")

    return V

In this case, only the value function must be returned.

Now, as it is necessary to have an initial policy, it can be create a function that initialize randomly.

In [23]:
def random_policy(grid):
    policy = {}

    for s in grid.actions.keys():
        policy[s] = np.random.choice(grid.actions[s])

    return policy

In [35]:
j = 0
for i in grid.actions.keys():
    j+= 1
    print("{}: {}".format(j,i))
print(len(grid.actions.keys()))

1: (0, 0)
2: (0, 1)
3: (0, 2)
4: (1, 0)
5: (1, 2)
6: (2, 0)
7: (2, 1)
8: (2, 2)
9: (2, 3)
9


In [27]:
random_policy(grid)

{(0, 0): 'R',
 (0, 1): 'R',
 (0, 2): 'D',
 (1, 0): 'U',
 (1, 2): 'D',
 (2, 0): 'R',
 (2, 1): 'L',
 (2, 2): 'R',
 (2, 3): 'U'}

In [26]:
type(random_policy(grid))

dict

### Example
With the function defined above it can create a random policy for the game.

In [16]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  →  |  →  |  ←  |     |
---------------------------
  ↑  |     |  ↓  |     |
---------------------------
  →  |  →  |  ↑  |  ↑  |


Now, the policy can be optimized on the board.

In [17]:
grid = create_grid()

V = optimal_policy(grid, policy, gamma=0.9)

print_value_policy(V, policy, grid)

Value function
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  →  |  ↑  |  ←  |


It can check how affects increasing the cost of doing an action up to a value of -2. For that, a new random policy is created.

In [18]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  →  |  ←  |  ↑  |  ←  |


Now it can optimize the policy on a board.

In [19]:
grid = create_grid(step_cost=-2)

V = optimal_policy(grid, policy, gamma=0.9)

print_value_policy(V, policy, grid)

Value function
---------------------------
-2.99|-1.10| 1.00| 0.00|
---------------------------
-4.69| 0.00|-1.00| 0.00|
---------------------------
-6.15|-4.61|-2.90|-1.00|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  →  |     |
---------------------------
  →  |  →  |  ↑  |  ↑  |


In this situation it is better for the agent to go towards the final state with a reward equal to -1 than towards the final state with reward +1.

## Optimizing the policy in a world with wind
An alternative to the basic game is a board with wind. In this a percentage of times the agent movements change randoly.The wind moves the player to an unwanted state. This can be implemented simply by entering a random part in the game.

In [20]:
def policy_evaluation_windy(grid, policy, V, states, windy=0.5, gamma=1, max_iter=100, threshold=1e-3):
    num_iter = 0

    while num_iter < max_iter:
        num_iter += 1
        biggest_change = 0
        for s in states:
            old_v = V[s]
            new_v = 0

            if s in policy:
                for action in grid.actions[s]:
                    if action == policy[s]:
                        p = windy
                    else:
                        p = (1-windy)/(len(grid.actions[s])-1)
                    grid.set_state(s)
                    r = grid.move(action)
                    new_v += p * (r + gamma * V[grid.current_state()])

                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))

        if biggest_change < threshold:
            break

    if num_iter >= max_iter:
        print("The maximum number of iterations has been reached")

In the policy optimization, this random movements must also be taken into account.

In [21]:
def optimal_policy_windy(grid, policy, windy=0.5, gamma=1, max_iter=100, threshold=1e-3):

    # Initialize the funciton value and states
    V, states = init_states(grid)

    # Iterate over policies until convergence or maximum iterations
    num_iter = 0

    while num_iter < max_iter:
        num_iter += 1

        # Policy evaluation
        policy_evaluation_windy(grid, policy, V, states,
                                windy, gamma, max_iter, threshold)

        # Improvement the policy
        is_policy_converged = True
        for s in states:
            if s in policy:
                old_a = policy[s]
                new_a = None
                best_value = float('-inf')

                # Iterate over actions until the best is obtained
                for action in Grid.possible_actions:
                    v = 0
                    for w_action in grid.actions[s]:
                        if action == w_action:
                            p = windy
                        else:
                            p = (1-windy)/(len(grid.actions[s])-1)
                        grid.set_state(s)
                        r = grid.move(action)
                        v += p * (r + gamma * V[grid.current_state()])
                    if v > best_value:
                        best_value = v
                        new_a = action
                policy[s] = new_a

                if new_a != old_a:
                    is_policy_converged = False

        if is_policy_converged:
            break

    if num_iter >= max_iter:
        print("The maximum number of iterations has been reached")

    return V

### Example
Now it is possible to repeat the previous exercise in this new world. A random policy is created.

In [22]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  ↓  |  →  |  →  |     |
---------------------------
  ↓  |     |  →  |     |
---------------------------
  →  |  ←  |  →  |  ←  |


Now it can be optimize the policy on a board with wind.

In [23]:
grid = create_grid()

V = optimal_policy_windy(grid, policy, gamma=0.9)

print_value_policy(V, policy, grid)

Value function
---------------------------
 0.20| 0.35| 0.57| 0.00|
---------------------------
 0.10| 0.00|-0.03| 0.00|
---------------------------
 0.01|-0.06|-0.16|-0.57|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  ←  |  ↑  |  ←  |


It can be checked that the policy changes when a penalty of -1 is added in each movement. To evaluate it, a new random policy is created.

In [24]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  ↓  |  ←  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  →  |  ←  |  ←  |


Now the policy can be optimized on a board with wind and a cost of -1 for any movement.

In [25]:
grid = create_grid(step_cost=-1)

V = optimal_policy_windy(grid, policy, gamma=0.9)

print_value_policy(V, policy, grid)

Value function
---------------------------
-5.92|-4.32|-1.47| 0.00|
---------------------------
-6.60| 0.00|-2.21| 0.00|
---------------------------
-6.53|-5.69|-3.89|-2.75|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  →  |     |
---------------------------
  →  |  →  |  ↑  |  ↑  |


Finally, as a check, if the wind equals 1, we have the results of the previous case. To evaluate it, a new random policy is created.

In [26]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  →  |  ←  |  ←  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  ←  |  ↑  |  ←  |


In [27]:
grid = create_grid(step_cost=-0.1)

V = optimal_policy_windy(grid, policy, gamma=0.9, windy=1)

print_value_policy(V, policy, grid)

Value function
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  →  |  ↑  |  ←  |


## Value function Optimization
Alternatively, the value function can be optimized. For this it can be  created the following function.

In [28]:
def optimal_value(grid, policy, gamma=0.9, max_iter=100, threshold=1e-3):
    V, states = init_states(grid)

    num_iter = 0
    while num_iter < max_iter:
        num_iter += 1
        biggest_change = 0

        for s in states:
            old_v = V[s]

            # La función de valor solo tiene valor si no es final
            if s in policy:
                new_v = float('-inf')
                for a in grid.actions[s]:
                    grid.set_state(s)
                    r = grid.move(a)
                    v = r + gamma * V[grid.current_state()]
                    if v > new_v:
                        new_v = v
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))

        if biggest_change < threshold:
            break

    if num_iter >= max_iter:
        print("The maximum number of iterations has been reached")

    # Obtener la politica para la función de valor
    for s in policy.keys():
        best_a = None
        best_value = float('-inf')
        # Itera sobre todas las posibles acciones
        for a in Grid.possible_actions:
            grid.set_state(s)
            r = grid.move(a)
            v = r + gamma * V[grid.current_state()]
            if v > best_value:
                best_value = v
                best_a = a

        policy[s] = best_a

    return V

### Example
As in the previous case, a random policy is used at the beginning.

In [29]:
policy = random_policy(grid)
print_policy(policy, grid)

---------------------------
  ↓  |  ←  |  →  |     |
---------------------------
  ↑  |     |  ↓  |     |
---------------------------
  →  |  →  |  ←  |  ↑  |


In [30]:
grid = create_grid(step_cost=-0.1)

V = optimal_value(grid, policy)

print_value_policy(V, policy, grid)

Value function
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|

Policy
---------------------------
  →  |  →  |  →  |     |
---------------------------
  ↑  |     |  ↑  |     |
---------------------------
  ↑  |  →  |  ↑  |  ←  |


The functions defined in this notebook are saved in a Python file to be imported and used in subsequent notebooks.